# 本セクションの目次
1. Avroフォーマット
2. 前方互換と後方互換と完全互換
3. メッセージキューとAvroを連携してみよう
4. Avroファイルの読み書き
5. Avroで前方互換をやってみよう

# PySparkとは？
PySparkとは分散処理を実現するエンジン(フレームワークの様なもの)です。  
ビッグデータの世界では、一つのノードで処理を行うということはほとんどなく複数台のノードで一つの仕事を行い結果を出すということがしばしば行われます。  

その処理を比較的簡単に行なってくれるのがSparkであり、Pythonとの組み合わせて使うことでPySparkと呼ばれています。  
他にはScalaとの組み合わせやJavaとの組み合わせが可能ですが、ビッグデータ分析を行うという観点からPythonが多く利用されています。

単純に複数台並べて多くのリクエストを捌く様なWebシステムというわけではなくて、一つの大きな仕事をを処理するためのフレームワークと考えると良いと思います。

# 分散処理とは？

分散処理とは、複数の端末にまたがって処理を行うことです。  
スレッド処理とは異なり複数台にまたがっていることがポイントです。

例えば、「This is good」「That are very good」のgoodの数を数えるプログラムを考えるときの場合を考えてみます。

## スレッド処理の場合
一台のパソコン（端末）の中でスレッドを立ち上げ処理を行います。

端末1  
　 - スレッドの呼び出しもと  
    - スレッド1 「This is good」のgoodの数を数える  
    - スレッド2 「That are very good」のgoodの数を数える  
-> スレッド１、２がそれぞれgoodの数を数えてスレッドの呼び出し元にてそれぞれのスレッドの合計を取得し結果を表示する

## 分散処理の場合
複数台(今回は3台)で処理を行います。

コントローラーノード   
　- 端末1,端末2に指示を出して処理をさせる  
　- 端末1,端末2からの処理結果を受け取りユーザに返却する  

端末1
  - 「This is good」のgoodの数を数える

端末2
  - 「That are very good」のgoodの数を数える

分散処理の特徴は、スレッド処理としてノードを(実質)無限にスケールすることが可能な点です。  
仮にCPUやメモリが足りなければ端末3,4,,,,と増やしていくことで処理のボトルネックを解決することが可能になるという理論です。  
一方で、スレッド処理は一つの端末内で実行されるため、無限にスケールすることができません(CPU/メモリに限界がある)  

今回のコースは、データも小さく端末が一台(コントローラーノードと端末1が同居している状態)ですが実際の環境で構築する(された)環境を利用する際は複数台であることが一般的です。

# ノートブックとは？

ノートブックとはGUIで視覚的に分析を可能にした、まさにノートの様な分析環境です。  
プログラムやこの様にマークダウンを記載する場所を「セル」と呼びます。  
セルにプログラムやその説明を記載しながら状況を残していきます。

分析の結果をノートの様に一個ずつ残していけるからノートブックと呼ばれています。

このノートブック自体は他の人に共有することも可能で、実行したグラフの結果などをそのまま表示させてGitにコミットを行うことによって  
共有を行います。

今回はVSCodeのノートブック環境を利用していますが他にも

- jupyter notebook
- EMR notebook
- zeppeline

などなど、クラウド環境で提供されているものもあります。

# Spark(PySpark)がデータ操作で利用するもの

Sparkがデータを操作するときに利用するものは2つ(正確には3つ)あります。  

- SQL
- DataFrame
- RDD

今回のコースは、SQLとDataFrameをそれぞれ対比させながら紹介をおこなっていこうと思います。  
また、あまり出番はないのですが、RDDについても少し触れてみようと思います。

SQLとはデータを読み込みし、そのデータに対して仮想的なテーブルを作成しSQLを使ってデータ操作を行います。  
DataFrameとはデータを読み込みし、そのデータに対してプログラムチックにデータ操作を行います。  
RDDとはデータを読み込みし、そのデータに対してプログラムチックにデータ操作を行います(DataFrameよりより、低レベルのAPIを提供します)。

Sparkには2つの読み込みタイプが存在しています  
- スキーマオンリード(事前のテーブル定義がなくてもデータを読み込んで処理が可能)  
- スキーマオンライト(データを読み込むためには、事前のテーブル定義が必要)  

既存のテーブルからデータを読み取ることも可能ですしなくても問題ありません。

詳しくはセクション3でみていきましょう。

# どのセクションでも登場する前準備

どのセクションでも、データ処理を開始する前のおまじないがあります。

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# Spark SessionはJavaでいうインスタンスの様なもので、spark(=分散処理しますよ)というマークだと考えればOK
# それ以外はconfigで細かな設定が可能
# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

# Jsonのデータ読み込み
SparkでJsonのデータを読み込んでみましょう。

In [ ]:
# Jsonデータの読み込み
json_df=spark.read.json("./dataset/jinko.json")

# CSVのデータ読み込み
CSV(TSV)の読み込みは非常にパターンが多いです。
ひとつを例にとってパターンを見てみましょう。

多すぎるので全て、紹介できませんがいくつかオプションをみていきましょう。

https://spark.apache.org/docs/latest/sql-data-sources-csv.html

- inferSchema(型をデータの中から推論するかどうか？推論なのでデータの中身によって毎度変わる可能性があるので注意)
- lineSep(sep),(csvであれば','でtsvであればタブ)
- header(一行目をカラム行としてみなすかどうか？)
- multiline(カラムで改行されたりした場合に、それを一行としてみるかどうか)
- Schema(スーキーマを設定する)
- encoding(読み込むエンコードを設定する)

In [ ]:
#データソースの読み込み
#sep='\t'とすればtsvでも読み込みが可能です
#multiLineは、CSVやTSVの各カラムに改行が含まれていた時の対策です。
df=spark.read.option("multiLine", "true").option("encoding", "SJIS").csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=True, sep=',', inferSchema=False)
df.show()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

#スキーマ設定をしていきましょう
struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])

df_csv=spark.read.option("multiLine", "true").option("encoding", "SJIS") \
    .csv("/home/pyspark/pyspark_super_crush_course/dataset/jinko.csv", header=False, sep=',', inferSchema=False, schema=struct)

df_csv.show(truncate=False,n=4)

# headerやsepなどはoptionで渡すことも可能ですが、csv関数の引数として渡すことも可能です。

# データフレーム
df_csvやjson_dfがデータフレームと呼ばれる変数

データを枠組み(フレーム)として扱うことができるのがデータフレームです。  
データフレームの作成方法は

- SQLから生成する
- データを読み込んで生成する
- プログラム的に作成する

SQLから生成はSQLで取得した結果がデータフレームで返却されるます  

データを読み込んで生成する場合はCSVやJSon、Parquet、Avroなどのファイルを読み込んでデータフレームに取り込みを行うことが可能  

プログラム的に作成するのは、空のデータフレームが欲しい時など

## データフレームの操作
データフレームに対する操作はたくさんあるのですが、今回はよく使う

- withColumns
- When
- null関係の操作(fillna)

について紹介していきたいと思います。

集計関数については別のセクションで紹介します。

In [ ]:
# 実演する

# カラムナーフォーマット
ここまでで読み込んだJsonやCsvはローデータとよばれる、データ基盤では扱いづらい部類に入るファイルです。  
扱いづらい理由はファイルがスプリッタブルかどうか？という点が関わっています。  

スプリッタブルであればデータを複数端末で分割して処理することが可能ですし、そうでなければ分割して処理をすることができないため非効率が発生します。  

カラムナーフォーマットと次のレクチャーで紹介するAvro(行指向フォーマット)はいずれもスプリッタブルなファイルです。  
そのため、CSVやJSON形式のままデータ基盤で活用を続けるのは悪手なので、必ずカラムナーフォーマット、行指向フォーマットへ変換する処理を挟みましょう。  

カラムナーフォーマットとは、分析に特化したファイル形式です。  
列ごとにデータをまとめて保存することによって、データを圧縮し検索速度(特にGroup byなどの集計構文)をあげています。  
また、カラムナーフォーマットは内部にカラム情報を保持しているためセクション2で紹介したスキーマオンリード機能を使ってスキーマ定義を読み出すことができます。

# 圧縮形式とファイルフォーマット

前のレクチャーではスプリッタブルかどうかが重要であるとお伝えしました。  
ここで一度、ファイルと圧縮形式の組み合わせについてみていきましょう。

```
        paruqet avro csv/json
gz       Y      Y      N
snappy   Y      Y      -
bz2      -      -      Y
圧縮なし  Y      Y      N

```

大事なのは、csvやjsonの場合はbz2を利用しましょう。

# 行指向フォーマット
先ほどはカラムナーフォーマットを紹介しましたが、次は行指向フォーマットを紹介について紹介していきます。  

よく利用されているRDBも行指向の仕組みを持ったデータシステムです。  
RDBはレコードの追加が得意です。  
 
そして、ビッグデータはレコードを一件つづ追加していく様な処理が苦手でしたが行指向フォーマットのAvroの登場によってレコードの追加が頻繁に発生する処理  
つまりストリーミング処理にも適用することが可能になりました。  

まず覚えてほしいのは、Avroはメインとしてストリーミング処理の利用に向いているということです（バッチ処理にも使うことが可能です）  


ビッグデータにおけるストリーミングについて詳しく知りたい方は  
「データサイエンスのためのストリーミング前処理入門　PythonとSparkで始めるビッグデータストリーミング処理入門」
を是非受講ください。

#　パーティションとダイナミックパーティション

データはパーティションと呼ばれれる区切りにデータを保存していくことが可能です。

```
テーブル
    パーティション1
        パーティション1に関するデータ

    パーティション2
        パーティション2に関するデータ
```

今回は、パーティションとしてkenmeiを利用して保存をしていこうと思います。
また、ファイルの出力としてParquet/Avroでそれぞれ出力を行ってみたいと思います。

## ダイナミックパーティション
ダイナミックパーティションとはデータをもとにパーティションを作成することを指します。  
Hiveなどで使われる言葉ですが、データをもとに振り分けるんだな。  
とだけ理解しておけばいいと思います。

In [ ]:
# repartitionはパーティション配下の
# パーティションなしの場合

# パーティションありの場合

# Avroでファイルを出力してみよう
avro_file.repartition(2).write.mode('overwrite').format("avro").save("/tmp/avro_etl/")

# output のパス
/home/pyspark/super_crush_course.db/parquet_table_with_partition
/home/pyspark/super_crush_course.db/parquet_table_with_no_partition

In [ ]:
spark.stop()